# Data Exploration

In [1]:
import pandas as pd
import numpy as np

# read files
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [2]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train, test], ignore_index=True)
print(train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
data.isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [4]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [5]:
#1.Item Visibility min value=0. Doesn't make sense as product being
#  sold in a store cannot have 0 visibility.
#2. Establishment Years need to be changed to how old store is for 
#  better impact on sales.

In [6]:
# Checking the nominal(categorical variables)
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
source                           2
dtype: int64

In [7]:
#This shows 1559 products, 10stores and 16 item types.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  11765 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  10188 non-null object
Outlet_Type                  14204 non-null object
source                       14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.4+ MB


In [9]:
# Filter categorical variables
categorical_columns = [x for x in data.dtypes.index
                       if data.dtypes[x] == 'object']
# Exclude ID cols and source:
categorical_columns = [x for x in categorical_columns if x not in
                       ['Item_Identifier', 'Outlet_Identifier', 'source']]
# Print frequency of categories
for col in categorical_columns:
    print('\nFrequency of Categories for varible %s' % col)
    print(data[col].value_counts())


Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

F

In [10]:
#Item Fat Contents are mis-coded as regular, LF and low fat.
#Too any item types, need to combine some to get better results.

# Data Cleaning

In [11]:
#Item Weight and Outlet size have null values. Fill it up.

In [12]:
# Impute data and check missing values before and after imputation to confirm
data.Item_Weight.fillna(data.Item_Weight.mean(), inplace=True)

In [13]:
data.head(3)

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,train


In [14]:
# Filling null values in Outlet Size with mode
data.Outlet_Size = data.groupby(['Outlet_Type'])['Outlet_Size'].apply(
    lambda x: x.fillna(x.mode()[0]))

### Modify Item_Visibility

In [15]:
#Treat the 0 item visibility as null and replace them with mean visibility
#  of each product.

In [16]:
data.Item_Visibility.replace(0,np.nan,inplace=True)

In [17]:
data.Item_Visibility = data.groupby(['Item_Identifier'])['Item_Visibility'].apply(lambda x: x.fillna(x.mean()))

### Create Broad Category of Items

In [18]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

### Years of Operation of Store

In [19]:
#Years:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

### Modify categories of Item Fat

In [20]:
#Change categories of low fat:
print ('Original Categories:')
print (data['Item_Fat_Content'].value_counts())

print ('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print (data['Item_Fat_Content'].value_counts())


Original Categories:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [21]:
#Mark non-consumables as separate category in low_fat:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

# Numerical and One hot Encoding of Categorical Variables

In [22]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [23]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet_Identifier'])

In [24]:
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
source                        object
Outlet_Years                   int64
Outlet                         int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_Identifier_OUT010       uint8
O

In [25]:
data.head()

,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,source,Outlet_Years,Outlet,...,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049
0,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,train,14,9,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,train,4,3,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,train,14,9,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,182.0950,732.3800,Fruits and Vegetables,0.022930,19.20,1998,train,15,0,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,53.8614,994.7052,Household,0.014670,8.93,1987,train,26,1,...,0,1,0,0,0,0,0,0,0,0


In [26]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [27]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [28]:
#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

# Model Building

In [30]:
train_data = pd.read_csv('train_modified.csv')
test_data = pd.read_csv('test_modified.csv')

In [31]:
#Define target and ID Columns
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet']

In [32]:
test_data.head()

,Item_Identifier,Item_MRP,Item_Visibility,Item_Weight,Outlet_Years,Outlet,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_0,...,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049
0,FDW58,107.8622,0.007565,20.750000,14,9,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,FDW14,87.3198,0.038428,8.300000,6,2,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,NCN55,241.7538,0.099575,14.600000,15,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,FDQ58,155.0340,0.015388,7.315000,6,2,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,FDY38,234.2300,0.118599,12.792854,28,5,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score, r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression
import statsmodels.api as sm

## Linear Regression Model

In [34]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression(normalize=True)

In [35]:
X_train = train_data.drop(['Item_Outlet_Sales','Item_Identifier'],axis=1)
y_train = train_data['Item_Outlet_Sales']

In [36]:
LR.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [37]:
X_test = test_data.drop(['Item_Identifier'],axis=1)

In [38]:
y_pred = LR.predict(X_train)

In [39]:
y_pred

array([4056.,  592., 2376., ..., 1472., 1432., 1216.])

In [40]:
score_lr = cross_val_score(LR,X_train,y_train,cv=5)

In [41]:
score_lr

array([0.57096248, 0.55518155, 0.54736229, 0.56678485, 0.56505486])

In [42]:
r2_score(y_train,y_pred)

0.5634233763396154

In [43]:
np.sqrt(mean_squared_error(y_train,y_pred))

1127.4854717178764

## Ridge Regression Model

In [44]:
from sklearn.linear_model import Ridge
RR = Ridge(alpha=0.05,normalize=True)

In [45]:
RR.fit(X_train,y_train)

Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [46]:
y_pred_rr = RR.predict(X_train)

In [47]:
y_pred_rr

array([3981.30411586,  640.29669277, 2375.65818407, ..., 1546.39836094,
       1446.6455034 , 1256.55229296])

In [48]:
r2_score(y_train,y_pred_rr)

0.5626391931307992

## Decision Tree Model

In [49]:
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)

In [50]:
DT.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=100,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [51]:
y_pred_dt = DT.predict(X_train)

In [52]:
y_pred_dt

array([4433.30403265,  711.06282087, 2309.200798  , ..., 1243.06892977,
       1372.68849074, 1103.18621333])

In [53]:
r2_score(y_train,y_pred_dt)

0.6143591048854546

## Random Forest Model

In [54]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(max_depth=8, min_samples_leaf=150)

In [55]:
RF.fit(X_train,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=150, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [56]:
y_pred_rf = RF.predict(X_train)

In [57]:
y_pred_rf

array([4127.05330248,  566.06345615, 2173.59631904, ..., 1389.44396847,
       1657.05814552, 1211.49115448])

In [58]:
r2_score(y_train,y_pred_rf)

0.6014882918381679